In [667]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from scipy.optimize import minimize
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import xgboost as xgb

import warnings 
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',200)

In [668]:
train= pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [669]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3865 entries, 0 to 3864
Data columns (total 80 columns):
galactic year                                                                              3865 non-null int64
galaxy                                                                                     3865 non-null object
existence expectancy index                                                                 3864 non-null float64
existence expectancy at birth                                                              3864 non-null float64
Gross income per capita                                                                    3837 non-null float64
Income Index                                                                               3837 non-null float64
Expected years of education (galactic years)                                               3732 non-null float64
Mean years of education (galactic years)                                                   3502 non-null 

In [670]:
train.describe(include='all')

,galactic year,galaxy,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,"Intergalactic Development Index (IDI), Rank",Population using at least basic drinking-water services (%),Population using at least basic sanitation services (%),Gross capital formation (% of GGP),"Population, total (millions)","Population, urban (%)","Mortality rate, under-five (per 1,000 live births)","Mortality rate, infant (per 1,000 live births)",Old age dependency ratio (old age (65 and older) per 100 creatures (ages 15-64)),"Population, ages 15–64 (millions)","Population, ages 65 and older (millions)","Life expectancy at birth, male (galactic years)","Life expectancy at birth, female (galactic years)","Population, under age 5 (millions)",Young age (0-14) dependency ratio (per 100 creatures ages 15-64),"Adolescent birth rate (births per 1,000 female creatures ages 15-19)",Total unemployment rate (female to male ratio),Vulnerable employment (% of total employment),"Unemployment, total (% of labour force)",Employment in agriculture (% of total employment),Labour force participation rate (% ages 15 and older),"Labour force participation rate (% ages 15 and older), female",Employment in services (% of total employment),"Labour force participation rate (% ages 15 and older), male",Employment to population ratio (% ages 15 and older),Jungle area (% of total land area),"Share of employment in nonagriculture, female (% of total employment in nonagriculture)",Youth unemployment rate (female to male ratio),"Unemployment, youth (% ages 15–24)","Mortality rate, female grown up (per 1,000 people)","Mortality rate, male grown up (per 1,000 people)","Infants lacking immunization, red hot disease (% of one-galactic year-olds)","Infants lacking immunization, Combination Vaccine (% of one-galactic year-olds)",Gross galactic product (GGP) per capita,"Gross galactic product (GGP), total","Outer Galaxies direct investment, net inflows (% of GGP)",Exports and imports (% of GGP),Share of seats in senate (% held by female),Natural resource depletion,"Mean years of education, female (galactic years)","Mean years of education, male (galactic years)","Expected years of education, female (galactic years)","Expected years of education, male (galactic years)","Maternal mortality ratio (deaths per 100,000 live births)",Renewable energy consumption (% of total final energy consumption),"Estimated gross galactic income per capita, male","Estimated gross galactic income per capita, female",Rural population with access to electricity (%),Domestic credit provided by financial sector (% of GGP),"Population with at least some secondary education, female (% ages 25 and older)","Population with at least some secondary education, male (% ages 25 and older)",Gross fixed capital formation (% of GGP),"Remittances, inflows (% of GGP)",Population with at least some secondary education (% ages 25 and older),Intergalactic inbound tourists (thousands),"Gross enrolment ratio, primary (% of primary under-age population)","Respiratory disease incidence (per 100,000 people)",Interstellar phone subscriptions (per 100 people),"Interstellar Data Net users, total (% of population)",Current health expenditure (% of GGP),"Intergalactic Development Index (IDI), female","Intergalactic Development Index (IDI), male",Gender Development Index (GDI),"Intergalactic Development Index (IDI), female, Rank","Intergalactic Development Index (IDI), male, Rank",Adjusted net savings,"Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total",Private galaxy capital flows (% of GGP),Gender Inequality Index (GII),y
count,3.865000e+03,3865,3864.000000,3864.000000,3837.000000,3837.000000,3732.000000,3502.000000,3474.000000,3474.000000,3432.000000,2021.000000,2015.000000,1502.000000,1271.000000,1271.000000,1271.000000,1259.000000,12

In [671]:
(train.isnull().sum()/len(train)*100)

galactic year                                                                               0.000000
galaxy                                                                                      0.000000
existence expectancy index                                                                  0.025873
existence expectancy at birth                                                               0.025873
Gross income per capita                                                                     0.724450
Income Index                                                                                0.724450
Expected years of education (galactic years)                                                3.441138
Mean years of education (galactic years)                                                    9.391979
Intergalactic Development Index (IDI)                                                      10.116429
Education Index                                                                            

In [672]:
train.head()


,galactic year,galaxy,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,"Intergalactic Development Index (IDI), Rank",Population using at least basic drinking-water services (%),Population using at least basic sanitation services (%),Gross capital formation (% of GGP),"Population, total (millions)","Population, urban (%)","Mortality rate, under-five (per 1,000 live births)","Mortality rate, infant (per 1,000 live births)",Old age dependency ratio (old age (65 and older) per 100 creatures (ages 15-64)),"Population, ages 15–64 (millions)","Population, ages 65 and older (millions)","Life expectancy at birth, male (galactic years)","Life expectancy at birth, female (galactic years)","Population, under age 5 (millions)",Young age (0-14) dependency ratio (per 100 creatures ages 15-64),"Adolescent birth rate (births per 1,000 female creatures ages 15-19)",Total unemployment rate (female to male ratio),Vulnerable employment (% of total employment),"Unemployment, total (% of labour force)",Employment in agriculture (% of total employment),Labour force participation rate (% ages 15 and older),"Labour force participation rate (% ages 15 and older), female",Employment in services (% of total employment),"Labour force participation rate (% ages 15 and older), male",Employment to population ratio (% ages 15 and older),Jungle area (% of total land area),"Share of employment in nonagriculture, female (% of total employment in nonagriculture)",Youth unemployment rate (female to male ratio),"Unemployment, youth (% ages 15–24)","Mortality rate, female grown up (per 1,000 people)","Mortality rate, male grown up (per 1,000 people)","Infants lacking immunization, red hot disease (% of one-galactic year-olds)","Infants lacking immunization, Combination Vaccine (% of one-galactic year-olds)",Gross galactic product (GGP) per capita,"Gross galactic product (GGP), total","Outer Galaxies direct investment, net inflows (% of GGP)",Exports and imports (% of GGP),Share of seats in senate (% held by female),Natural resource depletion,"Mean years of education, female (galactic years)","Mean years of education, male (galactic years)","Expected years of education, female (galactic years)","Expected years of education, male (galactic years)","Maternal mortality ratio (deaths per 100,000 live births)",Renewable energy consumption (% of total final energy consumption),"Estimated gross galactic income per capita, male","Estimated gross galactic income per capita, female",Rural population with access to electricity (%),Domestic credit provided by financial sector (% of GGP),"Population with at least some secondary education, female (% ages 25 and older)","Population with at least some secondary education, male (% ages 25 and older)",Gross fixed capital formation (% of GGP),"Remittances, inflows (% of GGP)",Population with at least some secondary education (% ages 25 and older),Intergalactic inbound tourists (thousands),"Gross enrolment ratio, primary (% of primary under-age population)","Respiratory disease incidence (per 100,000 people)",Interstellar phone subscriptions (per 100 people),"Interstellar Data Net users, total (% of population)",Current health expenditure (% of GGP),"Intergalactic Development Index (IDI), female","Intergalactic Development Index (IDI), male",Gender Development Index (GDI),"Intergalactic Development Index (IDI), female, Rank","Intergalactic Development Index (IDI), male, Rank",Adjusted net savings,"Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total",Private galaxy capital flows (% of GGP),Gender Inequality Index (GII),y
0,990025,Large Magellanic Cloud (LMC),0.628657,63.125200,27109.234310,0.646039,8.240543,NaN,NaN,NaN,NaN,NaN,NaN,38.305483,681.841086,22.131436,150.259290,129.841789,12.403812,490.446400,60.121580,57.028183,69.825369,46

# Data Preprocessing,Feature Engineering,Feature Selection and Modelling

### Converting categorical feature to numbers

In [673]:
df=pd.concat([train,test], sort=False)
df["galaxy"] = df["galaxy"].astype('category')
df["galaxy"] = df["galaxy"].cat.codes
train = df[:3865]
test = df[3865:]
test=test.drop("y", axis = 1)
test_res= test.copy()

### Checking how many galaxies are there and how many of them are distinct.

In [674]:
train_gal=set(train["galaxy"])
s=0
for x in train_gal:
    s=s+len(train.loc[train['galaxy'] == x])
print("Total distinct galaxies: {}".format(len(train_gal)))
print("Average samples per galaxy: {}".format(s/len(train_gal)))

Total distinct galaxies: 181
Average samples per galaxy: 21.353591160220994


In [675]:

test_gal=set(test["galaxy"])
s=0
for x in test_gal:
    s=s+len(test.loc[test['galaxy'] == x])
print("Total distinct galaxies: {}".format(len(test_gal)))
print("Average samples per galaxy: {}".format(s/len(test_gal)))

Total distinct galaxies: 172
Average samples per galaxy: 5.174418604651163



# Methods for Cross-validating Training Data

I trained a model for exery distinct galaxy in the training set (180) except the one from 126th galaxy as it has only one sample.

I used features with top 20 correlation with respect to y (target variable) galaxy specific.

Missing values are filled with the galaxy specific iterative imputer using BayesianRidge alghorithm.

Standard Scaler is used to standardize data.

XGBoost Regression is used as a model.

I found the paratameters by trying them manually one by one.

Both training and cross-validating performed galaxy-specific.

In [764]:
def cross_validation_loop(data,cor):
    labels= data['y']
    data=data.drop('galaxy', axis=1)    
    data=data.drop('y', axis=1)
    
    correlation=abs(data.corrwith(labels))
    columns=correlation.nlargest(cor).index
    data=data[columns]
    
    imp = IterativeImputer(max_iter=10,random_state=123)
    data=imp.fit_transform(data)

    scaler = StandardScaler().fit(data)
    data = scaler.transform(data)
        
    estimator =  xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.4,
                 n_estimators = 1000,n_jobs=-1,learning_rate=0.072,max_leaves=10,max_depth=3,subsample=0.8,random_state=123)
    
    cv_results = cross_validate(estimator, data, labels, cv=4, scoring='neg_root_mean_squared_error')

    error=np.mean(cv_results['test_score'])
    
    return error

Code for cross-validating a model for every galaxy

In [765]:
train_gal=set(train["galaxy"])
train_gal.remove(126)
def loop_train(cor):
    errors=[]
    for gal in train_gal:
        index = train.index[train['galaxy'] == gal]
        data = train.loc[index]
        errors.append(cross_validation_loop(data,cor))
    return np.mean(errors)

In [679]:
print(loop_train(20))

-0.006051210426390182


## Making predictions on the test data

Similar methodology is used to fill the missing value and standardization.

The bestthreshold in the cross validation, 20, is used.

And predicting performed based on galaxy-specific data.

In [778]:
def test_loop(data, test_data):
    labels= data['y']
    data=data.drop('galaxy', axis=1)    
    data=data.drop('y', axis=1)
    correlation=abs(data.corrwith(labels))
    columns=correlation.nlargest(20).index
    
    train_labels= labels
    train_data=data[columns]
    test_data= test_data[columns]
    
    imp =  IterativeImputer(max_iter=10,random_state=123).fit(train_data)
    train_data=imp.transform(train_data)
    test_data=imp.transform(test_data)

    scaler = StandardScaler().fit(train_data)
    train_data = scaler.transform(train_data)
    test_data = scaler.transform(test_data)

    model =  xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.4,
                 n_estimators = 1000,n_jobs=-1,learning_rate=0.072,max_leaves=10,max_depth=3,subsample=0.8,random_state=123)
    model.fit(train_data, train_labels)

    predictions = model.predict(test_data)
    return predictions

In [779]:
test=test_res
test=test.sort_values(by=['galaxy'])
test_pred = pd.DataFrame(0, index=np.arange(len(test)), columns=["predicted_y"])

In [780]:
i=0
for gal in test_gal:
    count=len(test.loc[test['galaxy'] == gal])
    index = train.index[train['galaxy'] == gal]
    data = train.loc[index]
    pred=test_loop(data,test.loc[test['galaxy']==gal])
    test_pred.loc[i:i+count-1,'predicted_y'] = pred
    i=i+count

# Optimization

I solved optimization problem by maxmizing allocation function.For maxmizing it we need to allocate max energy to galaxies with highest potential increase within the given constraints.

In [781]:
predictions = test_pred["predicted_y"]

In [782]:
predictions.head()

0    0.028422
1    0.038526
2    0.038526
3    0.036476
4    0.038526
Name: predicted_y, dtype: float64

In [800]:
def optimization(file_name):
    
    #Finding potential and likely increase 
    potential=pd.DataFrame({'potential':-np.log(predictions+0.01)+3})
    likely=potential**2/1000
    likely=likely.rename(columns={'potential': 'likely'})
    
    #Finding top 500 galaxies with highest potential increase
    top_idx=likely.sort_values(ascending=False,by='likely').index
    top_idx=top_idx[:500]
    
    
    #Finding galaxies that have existence expectancy index below 0.7
    low_idx=test[test.iloc[:,2]<0.7].index
    #Finding galaxies that have highest potential within these galaxies.
    low_idx=likely.loc[low_idx].sort_values(ascending=False,by='likely').index
    
    #creating submission df
    submission = pd.DataFrame({
    'pred': predictions,
    'opt_pred':0
    })
    
    #Finding galaxies that are both in highest potential galaxies and low existence expectancy galaxies with highest potential
    indices=[]
    count=0
    
    for i in low_idx:
        for a in top_idx:
            if i == a:
                indices.append(i)
                count+=1
                
                
    #Finding galaxies that allocated 0 energy  with low existence expectancy index           
    remaining=np.setdiff1d(low_idx,indices)
    
    
    if count >= 50: #All leei galaxies got energy
        test_pred.loc[top_idx,'opt_pred']=100
    else: #There are galaxies remained with 0 energy
        
        top_remaining=50-count #LEII galaxies must get total 5000 energy(50*100)
        
        #Finding remained galaxies and allocate energy to them
        top_gals=len(top_idx)-top_remaining
        top_idx=top_idx[0:top_gals]
        submission.loc[top_idx,'opt_pred']=100
        
        top_remaining=remaining[0:top_remaining]
        submission.loc[top_remaining,'opt_pred']=100
        
    
    path=r"C:\\Users\\Kjaerbaye\\Desktop\\ML&DS\\prohack\\"+ file_name + r".csv"
    submission.to_csv(path_or_buf=path)
    
    print('Sum of allocated energy: %s'%submission['opt_pred'].sum())
    print('Sum of allocated energy to the galaxies with leei < 0.7: %s'%submission['opt_pred'].loc[low_idx].sum())

In [801]:
optimization('submission')

Sum of allocated energy: 50000
Sum of allocated energy to the galaxies with leei < 0.7: 5000
